In [1]:
import os
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from numpy import random
from Levenshtein import distance
import sys
import pickle
from sklearn.cross_validation import train_test_split, KFold

In [15]:
#Read in Mayo documents
Mayo_list = []
for article in os.listdir('/users/markregalla/desktop/metis/Project4/MayoArticles'):
    with open("/users/markregalla/desktop/metis/Project4/MayoArticles/" 
              + article, 'r') as myfile:
        stuff = myfile.read()
    myfile.close
    if stuff != '':
        Mayo_list.append(stuff)
print len(Mayo_list)

35


In [16]:
#Read in Homeopathic documents
Homeo_list = []
k = 0
for article in os.listdir('/users/markregalla/desktop/metis/Project4/HomeoArticles'):
    with open('/users/markregalla/desktop/metis/Project4/HomeoArticles/' 
              + article, 'r') as myfile:
        stuff = myfile.read()
    myfile.close
    if stuff != '':
        Homeo_list.append(str(stuff.decode("ascii", "ignore")))
    k += 1
    if k == 35:  #match the number of Mayo Clinic articles
        break
print len(Homeo_list)

35


In [62]:
vectorizer = CountVectorizer(ngram_range=(1,2))
vectorizer.fit(Mayo_list)
x = vectorizer.transform(Mayo_list)
#'humans' in vectorizer.get_feature_names()
print x

  (0, 39)	1
  (0, 40)	1
  (0, 74)	9
  (0, 75)	1
  (0, 76)	1
  (0, 77)	1
  (0, 78)	5
  (0, 79)	1
  (0, 80)	1
  (0, 82)	1
  (0, 246)	1
  (0, 250)	1
  (0, 265)	1
  (0, 267)	1
  (0, 272)	1
  (0, 274)	1
  (0, 275)	1
  (0, 289)	1
  (0, 291)	9
  (0, 297)	1
  (0, 312)	1
  (0, 347)	1
  (0, 371)	1
  (0, 402)	1
  (0, 405)	1
  :	:
  (34, 4845)	1
  (34, 4848)	1
  (34, 4932)	3
  (34, 4966)	2
  (34, 4968)	1
  (34, 5022)	1
  (34, 5025)	1
  (34, 5056)	2
  (34, 5057)	1
  (34, 5058)	1
  (34, 5090)	1
  (34, 5091)	1
  (34, 5132)	1
  (34, 5133)	1
  (34, 5135)	2
  (34, 5142)	2
  (34, 5146)	1
  (34, 5148)	1
  (34, 5158)	1
  (34, 5159)	1
  (34, 5174)	3
  (34, 5175)	2
  (34, 5176)	1
  (34, 5322)	1
  (34, 5324)	1


In [65]:
x_array = x.toarray()
df = pd.DataFrame(x_array, columns=vectorizer.get_feature_names())

In [66]:
df

,000,000 years,10,10 minutes,10 percent,14,14 days,1886,1886 reportedly,1990s,...,yeast in,yeast infections,yeast legumes,yeast meat,yeast mushrooms,yellow,yellow fat,yellow vegetables,yogurt,yogurt other
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,1,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [57]:
#Trains on first 30 documents from each source, then finds Levenshtein 
#distances of last 5 documents from each source

Mayo_train_corpus = ''
Homeo_train_corpus = ''

#Build a training corpus for both sources
i = 0
while i < len(Mayo_list):
    Mayo_train_corpus = Mayo_train_corpus + Mayo_list[i]
    i += 1

i = 0
while i < len(Homeo_list):
    Homeo_train_corpus = Homeo_train_corpus + Homeo_list[i]
    i += 1
'''
#Provide report on comparison of training documents
print 'Mayo test to Mayo training corpus distance:'
for i in range(30,35):
    dist = distance(Mayo_train_corpus, Mayo_list[i])
    print 'Test Distance ' + str(i) + ': ' + str(dist)

print ''

print 'Homeo test to Homeo training corpus distance:'
for i in range(30,35):
    dist = distance(Homeo_train_corpus, Homeo_list[i])
    print 'Test Distance ' + str(i) + ': ' + str(dist)

print ''

print 'Homeo test to Mayo training corpus distance:'
for i in range(30,35):
    dist = distance(Mayo_train_corpus, Homeo_list[i])
    print 'Test Distance ' + str(i) + ': ' + str(dist)

print ''

print 'Mayo test to Homeo training corpus distance:'
for i in range(30,35):
    dist = distance(Homeo_train_corpus, Mayo_list[i])
    print 'Test Distance ' + str(i) + ': ' + str(dist)
'''

"\n#Provide report on comparison of training documents\nprint 'Mayo test to Mayo training corpus distance:'\nfor i in range(30,35):\n    dist = distance(Mayo_train_corpus, Mayo_list[i])\n    print 'Test Distance ' + str(i) + ': ' + str(dist)\n\nprint ''\n\nprint 'Homeo test to Homeo training corpus distance:'\nfor i in range(30,35):\n    dist = distance(Homeo_train_corpus, Homeo_list[i])\n    print 'Test Distance ' + str(i) + ': ' + str(dist)\n\nprint ''\n\nprint 'Homeo test to Mayo training corpus distance:'\nfor i in range(30,35):\n    dist = distance(Mayo_train_corpus, Homeo_list[i])\n    print 'Test Distance ' + str(i) + ': ' + str(dist)\n\nprint ''\n\nprint 'Mayo test to Homeo training corpus distance:'\nfor i in range(30,35):\n    dist = distance(Homeo_train_corpus, Mayo_list[i])\n    print 'Test Distance ' + str(i) + ': ' + str(dist)\n"

In [ ]:
'''
strategery: make seperate mayo and homeo training corpora
find distance of each document in that corpus to the corpus
average these distances
test document should be closer to that average distance for proper classification
'''

In [109]:
#Generate lists of average of each document's distance to the corpus
#Takes a while, unpickle if you need the results again
Mayo_dist_list = []
for i in range(30):
    dist = distance(Mayo_list[i], Mayo_train_corpus)
    Mayo_dist_list.append(dist)
    sys.stdout.write("\r" + str(i)) 

Homeo_dist_list = []
for i in range(30):
    dist = distance(Homeo_list[i], Homeo_train_corpus)
    Homeo_dist_list.append(dist)
    sys.stdout.write("\r" + str(i))

print Mayo_dist_list
print Homeo_dist_list

29[31762, 32842, 32321, 31798, 31877, 32277, 32009, 32712, 32619, 32019, 32744, 32219, 32206, 32631, 32475, 32489, 32797, 32558, 32668, 32366, 32661, 32614, 32401, 31961, 32425, 31991, 32174, 32003, 31554, 32384]
[609750, 636507, 642182, 641214, 608314, 596542, 603784, 594998, 620989, 630450, 633913, 637884, 646573, 637786, 625765, 633835, 624046, 603079, 635580, 640803, 614875, 588007, 638211, 642569, 635401, 622275, 645459, 637698, 636684, 630684]


In [125]:
Mayo_mean = np.mean(Mayo_dist_list, dtype=int)
Homeo_mean = np.mean(Homeo_dist_list, dtype=int)

In [61]:
#Classify remaining documents to see if it is close to appropriate average
def doc_classifier(corpus1, corpus2, x_train_1, x_train_2, y_train_1, y_train_2,
                   x_test, y_test, dist1_mean=0, dist2_mean=0):
    assert type(corpus1), type(corpus2) == 'str'
    assert type(x_train_1), type(x_train_2) == 'list'
    assert type(y_train_1), type(y_train_2) == 'list'
    assert type(x_test), type(y_test) == 'list'
    assert len(x_train_1) == len(x_train_2)
    
    #Find average of distances of x_train_1 to corpus1
    if dist1_mean == 0 and dist2_mean == 0:
        for i in range(len(x_train_1)):
            dist_1 = distance(corpus1, x_train_1[i])
            dist1_mean = dist1_mean + dist_1
            sys.stdout.write("\r" + '' + str(i))
            dist_2 = distance(corpus2, x_train_2[i])
            dist2_mean = dist1_mean + dist_2
            sys.stdout.write("\r" + 'Average ' + str(i) + ' of ' + str(len(x_train_1) - 1)
                             + ' completed.')
        dist1_mean = dist1_mean / len(x_train_1)
        dist2_mean = dist2_mean / len(x_train_2)
    
    result_list = [['Known', 'Predicted']]
    for i in range(len(x_test)):
        test_dist_1 = distance(corpus1, x_test[i])
        test_dist_2 = distance(corpus2, x_test[i])
        if abs(test_dist_1 - dist1_mean) < abs(test_dist_2 - dist2_mean):
            result_list.append([y_test[i], 'From Corpus 1'])
        else:
            result_list.append([y_test[i], 'From Corpus 2'])
        sys.stdout.write("\r" + 'Test ' + str(i) + ' of ' + str(len(x_test) - 1)
                         + ' completed.')
    
    return result_list
    
    '''    
        if abs(Mayo_dist - Mayo_mean) < abs(Homeo_dist - Homeo_mean):
            print 'Mayo Clinic source'
        else:
            print 'Homeopathe Intl source'
        print 'Test Distance ' + str(i) + ': ' + str(Mayo_dist)

    print ''
    '''

In [138]:
#Test/train split on documents
x = Mayo_list
print type(x)
y = ['Mayo' for x in range(len(Mayo_list))]
print type(y)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
print x_train, x_test, y_train, y_test

<type 'list'>
<type 'list'>


TypeError: Singleton array array(34) cannot be considered a valid collection.

In [139]:
#Test/train split on documents
kf = KFold(35, n_folds=7)
for train_index, test_index in kf:
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    #results = model.predict(x_test)
    #print type(results)
    #print results
    #print y_test.transpose()
    print list(y_test)


TypeError: 'int' object has no attribute '__getitem__'

In [52]:
##### Test/train split (my version)
#Create random indeces.  Don't let rand_array be run again to repeat model results
rand_array = np.random.choice(35,35, replace=False)

#28 for training, 7 for testing
x_train_Mayo = [Mayo_list[x] for x in rand_array[:28]]
y_train_Mayo = ['Mayo' for x in range(len(x_train_Mayo))]
x_train_Homeo = [Homeo_list[x] for x in rand_array[:28]]
y_train_Homeo = ['Homeo' for x in range(len(x_train_Homeo))]
print len(x_train_Homeo), len(y_train_Homeo)

#Generate test sets and appropriate labels
x_test = []
y_test = []
i = len(rand_array[:28])
while i < len(rand_array):
    x_test.append(Mayo_list[rand_array[i]])
    x_test.append(Homeo_list[rand_array[i]])
    y_test.append('Mayo')
    y_test.append('Homeo')
    i += 1

print len(x_train_Mayo), len(x_test), len(y_train_Mayo), len(y_test)
print x_test #, len(y_test)

28 28
28 14 28 14
['Vitamin B3 is made up of niacin and niacinamide, and can be found in many foods, including yeast, meat, fish, milk, eggs, green vegetables, and cereal grains. Vitamin B3 is often found in combination with other B vitamins, including thiamine, riboflavin, pantothenic acid, pyridoxine, cyanocobalamin, and folic acid.The U.S. Food and Drug Administration (FDA) has approved niacin for use in treating vitamin B3 deficiency (pellagra), which includes symptoms of skin inflammation, dementia, and diarrhea.Human research has shown that niacin is effective and relatively safe for treating high cholesterol levels. Limited evidence shows that niacin may help with clogged arteries and heart disease. Side effects commonly related to higher doses of niacin are itching, flushing, and stomach upset. Niacin may also cause liver problems, increased blood sugar, and hormone changes.', 'Tresorie - A PLEA FOR EARLY OPERATION IN PLEURISY WITH EFFUSION [Early\nOperation In Pleurisy With Ef

In [62]:
%time
result_list = doc_classifier(Mayo_train_corpus, Homeo_train_corpus, x_train_Mayo,
                             x_train_Homeo, y_train_Mayo, y_train_Homeo, x_test, y_test)
result_list

Test 13 of 14 completed.

[['Known', 'Predicted'],
 ['Mayo', 'From Corpus 1'],
 ['Homeo', 'From Corpus 1'],
 ['Mayo', 'From Corpus 1'],
 ['Homeo', 'From Corpus 1'],
 ['Mayo', 'From Corpus 1'],
 ['Homeo', 'From Corpus 1'],
 ['Mayo', 'From Corpus 1'],
 ['Homeo', 'From Corpus 1'],
 ['Mayo', 'From Corpus 1'],
 ['Homeo', 'From Corpus 1'],
 ['Mayo', 'From Corpus 1'],
 ['Homeo', 'From Corpus 1'],
 ['Mayo', 'From Corpus 1'],
 ['Homeo', 'From Corpus 1']]

###Pickling

In [111]:
with open('Mayo_dist_list.pkl', 'w') as picklefile:
    pickle.dump(Mayo_dist_list, picklefile)

In [112]:
with open('Homeoo_dist_list.pkl', 'w') as picklefile:
    pickle.dump(Homeo_dist_list, picklefile)